In [ ]:
# if using Google Colab - to connect to drive

# from google.colab import drive
# drive.mount('/content/google_drive')

In [ ]:
!sudo apt-get install cdo &> /dev/null

!pip install pyshp &> /dev/null
!pip install geopandas==0.13.0 &> /dev/null
#!pip install cftime &> /dev/null
#!pip install netcdf4 &> /dev/null

# load in libraries
import os
import sys
import pandas as pd
import numpy as np
import shapefile
import geopandas as gpd
from shapely.geometry import mapping
import shutil
import zipfile
from glob import glob
import xarray as xr

In [ ]:
# define paths
main_dir = "/content"
folder_name = "Milk_Mary_model"
drive_dir = os.path.join(main_dir, 'google_drive', 'MyDrive', folder_name)

Optional** - Prepare the shapefile for CaSPAr by altering the shapefile to a single polygon geometry

In [ ]:
# set up directories
# define temporary directory
net_dir = os.path.join(main_dir, 'data_temporary')
net_path = os.path.isdir(net_dir)
if not net_path:
  os.makedirs(net_dir)
  print("created  folder: ", net_dir)

# define temporary shapefile directory
shp_dir = os.path.join(main_dir, 'shapefile_temporary')
shp_path = os.path.isdir(shp_dir)
if not shp_path:
  os.makedirs(shp_dir)
  print("created  folder: ", shp_dir)

In [ ]:
# format shapefile by removing multipolygons
# Copyright 2016-2020 Juliane Mai - juliane.mai(at)uwaterloo.ca
# License
# This file is part of Juliane Mai's personal code library.
from __future__ import print_function
def coords2shapefile(filename,coords):

    # make sure coords is a list of lists
    coords = [ list(ii) for ii in coords ]

    # -----------------------
    # Check if polygon is clockwise:
    #       Use "shapefile.signed_area()" method to determine if a ring is clockwise or counter-clockwise
    #       Value >= 0 means the ring is counter-clockwise.
    #       Value <  0 means the ring is clockwise
    #       The value returned is also the area of the polygon.
    # -----------------------
    area = shapefile.signed_area(coords)

    if area >= 0:
        coords.reverse()    # transform counter-clockwise to clockwise

    if sys.version_info < (3,0,0):
        # ------------------------
        # Create a polygon shapefile
        # ------------------------
        # Found under:
        #     https://code.google.com/archive/p/pyshp/
        w = shapefile.Writer(shapefile.POLYGON)

        # an arrow-shaped polygon east of Vancouver, Seattle, and Portland
        w.poly([coords])
        w.field('FIRST_FLD','C','40')
        w.record('First','Polygon')
        w.save(filename)
    else:
        # ------------------------
        # Create a polygon shapefile
        # ------------------------
        # Found under:
        #     https://code.google.com/archive/p/pyshp/
        w = shapefile.Writer(target=filename)

        # an arrow-shaped polygon east of Vancouver, Seattle, and Portland
        w.poly([coords])
        w.field('FIRST_FLD','C','40')
        w.record('First','Polygon')
        w.close()


    # ------------------------
    # Write projection information
    # ------------------------
    # Found under:
    #     https://code.google.com/archive/p/pyshp/wikis/CreatePRJfiles.wiki
    prj = open("%s.prj" % filename, "w")
    epsg = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]'
    prj.write(epsg)
    prj.close()

    return

# read in shapefile and check geometry
"""for shp_file in os.listdir(os.path.join(drive_dir, 'shapefile')):    # finds shapefile name for you, however only useful if there is only one shapefile in folder
    if shp_file.endswith(".shp"):
      shp_file_name = shp_file
shp_file = gpd.read_file(os.path.join(drive_dir, 'shapefile', shp_file_name))"""

shp_file = gpd.read_file(os.path.join(drive_dir, 'shapefile', 'studyArea_outline.shp'))

# format shapefile if not a singular polygon
explode = shp_file.explode()
num_poly = explode['geometry'].count()

if num_poly > 1:
  g = [i for i in shp_file.geometry]
  geojson_ob = mapping(g[0]) # for first feature/row
  all_coords = geojson_ob["coordinates"]

  print(all_coords)

  flat_list = []

  for sublist in all_coords[0]:
      for item in sublist:
          flat_list.append(item)

  if __name__ == '__main__':
    # import doctest
    # doctest.testmod(optionflags=doctest.NORMALIZE_WHITESPACE)
    coords = flat_list
    filename = os.path.join(shp_dir,'polygon')   # don't use any file ending here

    coords2shapefile(filename,coords)

    shp_check = gpd.read_file(os.path.join(shp_dir,'polygon.shp'))
    shp_check.plot()

else:
  shp_file = gpd.read_file(os.path.join(drive_dir, 'shapefile', 'studyArea_outline.shp'))    # may need to adjust shapefile name
  shp_file.to_file(os.path.join(shp_dir, 'study_area.geojson'), driver='GeoJSON')
  shp_file.plot()

In [ ]:
# zip folder
zip_name = os.path.join(shp_dir)
directory_name = os.path.join(main_dir, 'shapefile')
print('Folder zipped')

# Create 'path\to\zip_file.zip'
shutil.make_archive(directory_name, 'zip', zip_name)

Right click on the zipped folder and select "Download" to save to your local computer

Once the zipped folder has been downloaded to your local computer, please click on the following link: [CaSPAr Data Portal](https://caspar-data.ca/caspar)

Need a Globus ID, no problem! Click [here](https://www.globusid.org/) for more information

If you are new to Globus you will also need to install Globus Connect Personal in order to transer the CaSPAr data to your local machine, the link to download the software is available [here](https://www.globus.org/globus-connect-personal)


In [ ]:
# define new directories
# define temporary directory
net_temp_dir = os.path.join(main_dir, 'data_temporary') # path to where the CaSPAr data is saved
net_temp_path = os.path.isdir(net_temp_dir)
if not net_path:
  os.makedirs(net_temp_dir)
  print("created  folder: ", net_temp_dir)

# define bash script directory
bash_dir = os.path.join(main_dir, 'data_temporary','bash_scripts')
bash_path = os.path.isdir(bash_dir)
if not bash_path:
  os.makedirs(bash_dir)
  print("created  folder: ", bash_dir)

# define CaSPAr output directory
caspar_dir = os.path.join(drive_dir, 'workflow_outputs','RavenInput', 'input')
caspar_path = os.path.isdir(caspar_dir)
if not caspar_path:
  os.makedirs(caspar_dir)
  print("created  folder: ", caspar_dir)

In [ ]:
# unzip CaSPAr data
extension = ".zip"

os.chdir(net_dir) # change directory from working dir to dir with files

for item in os.listdir(net_temp_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(net_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped

In [ ]:
# define merged CaSPAr data name
# note - do NOT add .nc on the end of the merged CaSPAr name
merged_Casp_name = 'RDRSv2_hourly'

In [ ]:
# overview of the the NetCDF files thus far
configfiles = glob('/content/data_temporary/*/*.nc') # may need to adjust path, depending where data is saved
test = xr.open_dataset(configfiles[0])
print(test)
#display(test)      # if in Jupyter Notebook environment

In [ ]:
# merge NetCDF files
# define path of CaSPAr file, do not include the name of the files
# for example, /content/data_temporary/data/

merged_Casp_name = 'caspar_data'

casp_file_path = '/content/data_temporary/hb_1688580282/'  # adjust path as needed

casp_file = os.path.join(casp_file_path,'*.nc')

#bash command
with open(os.path.join(bash_dir,'merge.sh'), 'w') as f2:
    print(f'cdo mergetime {casp_file} /content/data_temporary/{merged_Casp_name}.nc', file=f2) # adjust path as needed

!bash /content/data_temporary/bash_scripts/merge.sh  # able to run bash directly in Jupyter notebook environment

In [ ]:
# adjust unit of time
# The data products from CaSPAr are in UTC time, meaning that the data will likely
# need to be shifted in order to align with other data in your model (such as observed flows)
ds = xr.open_dataset('/content/data_temporary/caspar_data.nc') # read in NetCDF

# Time shift to local time zone
UTC_tz = 5      # UTC-5
old_start_datetime = ds['time'][0].to_numpy()[()]
new_start_datetime = old_start_datetime - np.timedelta64(UTC_tz, 'h')
utc5time = pd.date_range(new_start_datetime, freq='1H', periods=len(ds['time']))
ds['time'] = utc5time

# save the merged netcdf file
ds.to_netcdf('/content/data_temporary/caspar_data_adjusted.nc', 'w') # adjust path as needed

In [ ]:
# overview of adjusted CaSPAr data
check_casp = xr.open_dataset(os.path.join(net_dir, f'caspar_data_adjusted.nc'))
print(check_casp)
# display(check_casp)  # if in Jupyter Notebook environment

In [ ]:
# temporatl aggregation of hourly data to daily

# PRECIPITATION -----------------------------------------------------------------------------------------------------------------------------------
# convert from m to mm
precip = check_casp['RDRS_v2.1_P_PR0_SFC']                                              # need to be customized by user
precip = precip * 1000
precip_ds = precip.to_dataset()
# save the aggregated file
precip_ds.to_netcdf(os.path.join(net_dir, 'prcp_mm_data.nc'), 'w')

# aggregation of hourly to daily data
# adjust path as needed
!cdo daysum /content/data_temporary/prcp_mm_data.nc /content/google_drive/MyDrive/Milk_Mary_model/workflow_outputs/RavenInput/input/mean_prcp_daily.nc


# TEMPERATURE -----------------------------------------------------------------------------------------------------------------------------------
temp = check_casp['RDRS_v2.1_P_TT_1.5m']
mean_temp_ds = temp.to_dataset()
mean_temp_ds.to_netcdf(os.path.join(net_dir, 'temp_data.nc'), 'w')
# aggregation of hourly to daily data

# Daily Average Temperature
# adjust path as needed
# first path - where the input data is saved
# second path - where the output data should be saved/named
!cdo daymean /content/data_temporary/temp_data.nc /content/google_drive/MyDrive/Milk_Mary_model/workflow_outputs/RavenInput/input/mean_temp_daily.nc

# Daily Maximum Temperature
# adjust path as needed
# first path - where the input data is saved
# second path - where the output data should be saved/named
!cdo daymax /content/data_temporary/temp_data.nc /content/google_drive/MyDrive/Milk_Mary_model/workflow_outputs/RavenInput/input/max_temp_daily.nc

# Daily Minimum Temperature
# adjust path as needed
# first path - where the input data is saved
# second path - where the output data should be saved/named
!cdo daymin /content/data_temporary/temp_data.nc /content/google_drive/MyDrive/Milk_Mary_model/workflow_outputs/RavenInput/input/min_temp_daily.nc

In [ ]:
# check NetCDF file again (ensure they are not empty)
check_casp = xr.open_dataset('/content/google_drive/MyDrive/Milk_Mary_model/workflow_outputs/RavenInput/input/min_temp_daily.nc')
print(check_casp)
#display(check_casp)  # if in Jupyter environment

In [ ]:
# generate list of filenames for Raven RVT file
file_names_lst = []
files_input = glob(os.path.join(caspar_dir, "*")) # may need to adjust path to where the final CaSPAr files are saved

for f_name in list(range(0, len(files_input))):
  base = os.path.basename(files_input[f_name])
  file_names_lst.append(base)

print(file_names_lst)

**Generate Raven RVT File** </font></br>

example format of a Gridded Forcing RVT file:<br>
<font color=grey> :GriddedForcing           Averagetemp<br>
><font color=grey> :ForcingType            TEMP_AVE<br>
>:FileNameNC             input/VIC_temperatures.nc<br>
>:VarNameNC              Avg_temp<br>
>:DimNamesNC             lon lat time<br>
>:RedirectToFile         GridWeights.txt

<font color=grey> :EndGriddedForcing

In [ ]:
# Define model name (for RVT file)
model_name = 'model_name'

In [ ]:
# Define forcing types
# include quotations around each variable and square brackets
# for example, ['Precipitation', 'Averagetemp']
forcing_name = ['Mintemp', 'Maxtemp', 'Precipitation']

In [ ]:
# Define forcing types
# include quotations around each variable and square brackets
# for example, ['PRECIP', 'TEMP_AVE']
forcing_type = ['TEMP_DAILY_MIN', 'TEMP_DAILY_MAX', 'PRECIP']

In [ ]:
# Define variable name in NetCDF file
# include quotations around each variable and square brackets
# for example, ['RDRS_v2.1_P_PR0_SFC', 'RDRS_v2.1_P_TT_1.5m']
var_names = ['RDRS_v2.1_P_TT_1.5m', 'RDRS_v2.1_P_TT_1.5m', 'RDRS_v2.1_P_PR0_SFC']

In [ ]:
# Define dimension names
# seperate with spaces and include quotations
# for example, 'rlon rlat time'
dim_names = 'rlon rlat time'

In [ ]:
# using a print statement to write and save the RVT file
with open(os.path.join(drive_dir, 'workflow_outputs','RavenInput',model_name+'.rvt'), "a") as f:
  print(f"#------------------------------------------------------------------------", file=f)
  print(f"# CaSPAr Data Input File", file=f)
  print(f"#------------------------------------------------------------------------\n#", file=f)

lst_vals = list(range(0, len(forcing_type)))

for n in lst_vals:
  #n1 = forc_input[(forc_input.lon == df_coords1.lon[n])&(forc_input.lat == df_coords1.lat[n])].reset_index(drop=True)
  #print(n1)
  f = open(os.path.join(drive_dir, 'workflow_outputs','RavenInput',model_name+'.rvt'), "a")
  print(f":GriddedForcing \t\t\t{forcing_name[n]}", file=f)
  print(f"\t:ForcingType \t\t\t\t{forcing_type[n]}", file=f)
  print(f"\t:FileNameNC \t\t\t\tinput/{file_names_lst[n]}", file=f)
  print(f"\t:VarNameNC \t\t\t\t\t{var_names[n]}", file=f)
  print(f"\t:DimNamesNC \t\t\t\t{dim_names}", file=f)
  print(f"\t:RedirectToFile \t\tGridWeights.txt", file=f)
  print(f":EndGriddedForcing", file=f)
  print(f"#", file=f)
  f.close()

In [ ]:
# optional** - remove temporary folders that were generated and no longer usefule
remove_drainage_region_temporary_files = False   # change to True to delete folders

# delete temorary folder
if remove_drainage_region_temporary_files == True:
  shutil.rmtree(os.path.join(main_dir, 'data_temporary'))